In [ ]:
# ==========================================================
# Maestría en Ciencia y Análisis de Datos
# Universidad Mayor de San Andrés
# ----------------------------------------------------------
#            Machine Learning y Deep Learning
# ----------------------------------------------------------
#         Rolando Gonzales Martinez, Agosto 2024
# ==========================================================
#        Redes neuronales (perceptron) multicapa
# ==========================================================
import numpy as np
import matplotlib.pyplot as plt

# Función de activación Sigmoide
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Derivada de la función Sigmoide
def sigmoid_derivative(z):
    return z * (1 - z)

# Función de pérdida de error cuadrático medio
def mse_loss(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# MLP simple con una capa oculta
class MLP:
    def __init__(self, input_size, hidden_size, output_size):
        # Inicialización de pesos aleatorios
        self.weights_input_hidden = np.random.randn(input_size, hidden_size)
        self.weights_hidden_output = np.random.randn(hidden_size, output_size)
        self.bias_hidden = np.random.randn(hidden_size)
        self.bias_output = np.random.randn(output_size)

    def forward(self, X):
        # Paso 1: Propagación hacia adelante
        self.z1 = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.a1 = sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.weights_hidden_output) + self.bias_output
        self.a2 = sigmoid(self.z2)
        return self.a2

    def backward(self, X, y, output, learning_rate):
        # Paso 2: Retropropagación
        # Error en la capa de salida
        output_error = y - output
        output_delta = output_error * sigmoid_derivative(output)

        # Error en la capa oculta
        hidden_error = np.dot(output_delta, self.weights_hidden_output.T)
        hidden_delta = hidden_error * sigmoid_derivative(self.a1)

        # Actualización de pesos y sesgos
        self.weights_hidden_output += np.dot(self.a1.T, output_delta) * learning_rate
        self.bias_output += np.sum(output_delta, axis=0) * learning_rate
        self.weights_input_hidden += np.dot(X.T, hidden_delta) * learning_rate
        self.bias_hidden += np.sum(hidden_delta, axis=0) * learning_rate

    def train(self, X, y, epochs, learning_rate):
        loss_history = []
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output, learning_rate)
            loss = mse_loss(y, output)
            loss_history.append(loss)
            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Loss: {loss}")
        return loss_history

# Datos de entrenamiento (XOR problem)
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])

# Crear una red neuronal con 2 entradas, 2 neuronas en la capa oculta, y 1 salida
mlp = MLP(input_size=2, hidden_size=2, output_size=1)

# Entrenar la red
epochs = 10000
learning_rate = 0.1
loss_history = mlp.train(X, y, epochs=epochs, learning_rate=learning_rate)

# Probar la red después del entrenamiento
output = mlp.forward(X)
print("Predicciones después del entrenamiento:")
print(output)

# Graficar la pérdida durante el entrenamiento
plt.figure(figsize=(10, 5))
plt.plot(range(epochs), loss_history, label='Loss')
plt.xlabel('Epocas (Epochs)')
plt.ylabel('Perdida (Loss, L)')
plt.title('Evolución de la Pérdida Durante el Entrenamiento')
plt.grid(True)
plt.legend()
plt.show()

# Graficar las predicciones finales y la línea de separación aprendida
x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                     np.arange(y_min, y_max, 0.01))

Z = mlp.forward(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.figure(figsize=(10, 5))
plt.contourf(xx, yy, Z, levels=[0, 0.5, 1], alpha=0.5, cmap=plt.cm.bwr)
plt.scatter(X[:, 0], X[:, 1], c=y.ravel(), s=100, edgecolors='k', marker='o', cmap=plt.cm.bwr)
plt.xlabel('Entrada 1')
plt.ylabel('Entrada 2')
plt.title('Espacio de Decisión Aprendido por el MLP para el problema XOR')
plt.grid(True)
plt.show()
